In [ ]:
import pandas as pd
df=pd.read_csv("kdrama.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  250 non-null    object 
 1   Aired Date            250 non-null    object 
 2   Year of release       250 non-null    int64  
 3   Original Network      250 non-null    object 
 4   Aired On              250 non-null    object 
 5   Number of Episodes    250 non-null    int64  
 6   Duration              250 non-null    object 
 7   Content Rating        245 non-null    object 
 8   Rating                250 non-null    float64
 9   Synopsis              250 non-null    object 
 10  Genre                 250 non-null    object 
 11  Tags                  250 non-null    object 
 12  Director              249 non-null    object 
 13  Screenwriter          249 non-null    object 
 14  Cast                  250 non-null    object 
 15  Production companies  2

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
data = pd.read_csv('kdrama.csv')
data = data.drop(['Name', 'Aired On', 'Aired Date', 'Content Rating', 'Synopsis', 'Tags', 'Director', 'Screenwriter', 'Cast', 'Production companies', 'Rank'], axis=1)

# Cleaning the data
data['Year of release'] = data['Year of release'].astype(str)
data['Original Network'] = data['Original Network'].apply(lambda x: x.strip())

# Extraction of features
tfidf = TfidfVectorizer(stop_words='english')
data['Genre'] = data['Genre'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['Genre'])

# Computing cosine similarity
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(data.index, index=data['Original Network']).drop_duplicates()

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Load the data
# df = pd.read_csv("kdrama.csv")

# # Select features for content-based filtering
# features = ["Name", "Synopsis", "Genre", "Tags", "Director", "Cast", "Production companies"]

# # Remove missing values
# df = df.dropna(subset=features)

# # Combine the features into a single string
# df["content"] = df[features].apply(lambda x: " ".join(x.astype(str)), axis=1)

# # Vectorize the content using TF-IDF
# tfidf = TfidfVectorizer(stop_words="english")
# tfidf_matrix = tfidf.fit_transform(df["content"])

# # Calculate the cosine similarity matrix
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing text
def preprocess_text(text):
    # Converting to lowercase
    text = text.lower()
    
    # Removing punctuation
    text = ''.join(c for c in text if c not in '.,?!-()[]{};:\'\"')
    
    return text

# Creating a TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess_text)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Synopsis'])

# Extraction of features from text
def extract_features(text, tfidf_matrix):
    # Preprocess text
    text = preprocess_text(text)
    
    # Transform text using TF-IDF vectorizer
    text_vector = tfidf_vectorizer.transform([text])
    
    return text_vector

# Recommend similar dramas
def recommend_similar_dramas(drama_name, dramas, tfidf_matrix):
    # Preprocessing input drama name
    drama_name = preprocess_text(drama_name)
    
    # Extraction of features for input drama name
    drama_features = extract_features(drama_name, tfidf_matrix)
    
    # Calculating similarity between input drama and all other dramas
    similarities = cosine_similarity(drama_features, tfidf_matrix)
    
    # Finds indices of top 5 similar dramas
    similar_dramas_indices = similarities.argsort()[0][-6:-1]
    
    # Returns names of top 5 similar dramas
    similar_dramas = list(dramas['Name'].iloc[similar_dramas_indices])
    
    return similar_dramas


In [ ]:
input_drama = 'Crash Landing on You'
similar_dramas = recommend_similar_dramas(input_drama, df, tfidf_matrix)
print(similar_dramas)


['The Mysterious Class', 'Crash Landing on You', 'Vagabond', 'Mad Dog', 'Extraordinary You']


In [ ]:
!pip install streamlit 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile app.py
import streamlit as st

st.write("Hello, World!")


Overwriting app.py


In [ ]:
!ngrok authtoken 2LQgLh7lodeoIYd3YbBA1TKICLR_UzSY2aEtgZgrVPJx8mTB

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok

NAME:
  ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

DESCRIPTION: 
  ngrok exposes local networked services behinds NATs and firewalls to the
  public internet over a secure tunnel. Share local websites, build/test
  webhook consumers and self-host personal services.
  Detailed help for each command is available with 'ngrok help <command>'.
  Open http://localhost:4040 for ngrok's web interface to inspect traffic.

Author:
  ngrok - <support@ngrok.com>

TERMS OF SERVICE: https://ngrok.com/tos

EXAMPLES: 
  ngrok http 80                           # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080  # port 8080 available at baz.ngrok.dev
  ngrok http foo.dev:80                   # tunnel to host:port instead of localhost
  ngrok http https://localhost            # expose a local https server
  ngrok tcp 22                            # tunnel arbitrary TCP traffic to port 22
  ngrok tls --domain=foo.com 44

In [ ]:
from pyngrok import ngrok

In [ ]:
!streamlit run app.py&>/dev/null&

In [ ]:
!pgrep streamlit 

10567


In [ ]:
pub_url = ngrok.connect(port='8501')

In [ ]:
pub_url

'http://f5fb-34-138-159-95.ngrok-free.app'

In [ ]:
%%writefile app.py 
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the data
df = pd.read_csv("kdrama.csv")

# Selecting features for content-based filtering
features = ["Name", "Synopsis", "Genre", "Tags", "Director", "Cast", "Production companies"]

# Removal of missing values
df = df.dropna(subset=features)

# Combining the features into a single string
df["content"] = df[features].apply(lambda x: " ".join(x.astype(str)), axis=1)

# Vectorization of the content using TF-IDF
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["content"])

# Calculating the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Streamlit app title 
st.title("KDrama Recommendation System")

# Display a dropdown menu with all drama names
options = list(df["Name"])
selected_drama = st.selectbox("Select a drama you liked:", options)

# Get the index of the selected drama
index = df.loc[df["Name"] == selected_drama].index[0]

# Calculate the pairwise similarity between the selected drama and all others
similarity_scores = list(enumerate(cosine_sim[index]))

# Sort the similarity scores in descending order
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Display the top 5 most similar dramas
st.write("Top 5 similar dramas:")
for i in range(1, 6):
    drama = df.iloc[similarity_scores[i][0]]["Name"]
    st.write(f"{i}. {drama}")

Overwriting app.py
